In [2]:
from src.utils import petab as pet
import numpy as np
import src.utils.sbml as sbml
import src.utils.amici as am
from src.models.sbml_models import CRP2_CPE
%load_ext autoreload
%autoreload 2

### Define SBML/AMICI Model

In [3]:
observables_df = pet.define_observables({
    'xA': 'xA',
    'xB': 'xB',  
}, noise_value=0.02)

# Load SBML/AMICI model
sbml_model_filepath = sbml.write_model(CRP2_CPE)

validator = sbml.validateSBML(ucheck=False)
validator.validate(sbml_model_filepath)

model = am.load_amici_model(sbml_model_filepath, observables_df=observables_df, force_compile=True)


### Define Measurements

In [3]:
# Define model parameters
true_params = {
    # 'kpAA': 1.0,
    "rA": 5.0,
    "rB": 1.0,
    "rX": 1.0,
    "KAA": 0.0,
    "KAB": 0.0,
    "KBA": 0.0,
    "KBB": 0.0,
}
model = am.set_model_parameters(model, true_params)

# Define conditions
fA0 = np.array([0.25, 0.5, 0.75], dtype=float)
cM0 = np.array([1.0, 1.0, 1.0], dtype=float)
A0 = fA0 * cM0
B0 = (1 - fA0) * cM0

# Define conditions dataframe
conditions_df = pet.define_conditions(
    init_conditions={
        "A0": list(A0),
        "B0": list(B0),
    }
)
print(conditions_df)

# Define conversion values
timepoints = list(np.arange(0, 1, 0.1, dtype=float))

amici_measurements_df, rdatas = pet.define_measurements_amici(
    amici_model=model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    debug_return_rdatas=True,
)

amici_measurements_df

### Define CPE Measurements

In [4]:
from src.models import cpe_models as CPE

cpe_model = CPE.Model.from_ratios(list(true_params.values()))
cpe_model.solve(fA0=0.75, M0=1.0, t_eval=timepoints, approach='wittmer')

# Not implemented yet!
cpe_measurements_df = pet.define_measurements_cpe(
    cpe_model=cpe_model,
    timepoints=timepoints,
    conditions_df=conditions_df,
    observables_df=observables_df,
    obs_sigma=0.0,
    meas_sigma=0.005,
    approach="izu",
)

cpe_measurements_df